## Scraping CNN

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [2]:
# Mengecek Link status Ok
url = "https://www.cnnindonesia.com/search/?query=ganjar&page=2"
response = requests.get(url)
response

<Response [200]>

In [3]:
jumlah_index = 1
threads_link = []
links = []
results = []
keywords = 'prabowo'

In [4]:
def scrape_links(page_number,keywords):
    try:
        
        url = f"https://www.cnnindonesia.com/search/?query={keywords}&page={page_number}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.find_all('article', {"class": "flex-grow"})
        
        page_links = []
        for article in articles:
              link_element = article.find('a')
              if link_element is not None:
                        link = link_element.get('href')
                        if link is not None:
                                page_links.append(link)
        
        print(f"Scraped {len(page_links)} links from page {page_number}")
        return page_links
    except Exception as e:
        print(f"Error while scraping page {page_number}: {str(e)}")
        return []

In [5]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))


Scraped 24 links from page 1
Total Links: 24


In [6]:
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            # Judul Berita
            title_elem = soup.find('h1', {"class": "mb-2 text-[28px] leading-9 text-cnn_black"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('span', {"class": "text-cnn_red"})
            if author_elem:
                author_text = author_elem.get_text()
                author_text = author_text.split('-')[0].strip()
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "text-cnn_grey text-sm mb-4"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('a', {"dtr-sec": "subkanal"})
            if category_elements:
                category_text= category_elements.text.strip()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "detail-text text-cnn_black text-sm grow min-w-0"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            nama_berita_match = re.search(r'https://(.*?)/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"
            results.append({'title': title_text,
                            'keywords':keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [7]:
threads = []
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [8]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 24


,title,keywords,author,category,date,content,link
0,"Tak Hanya Timor Leste, Kepulauan Solomon juga ...",prabowo,CNN Indonesia,Asia Pasifik,"Rabu, 27 Sep 2023 08:17 WIB","Kepulauan Solomon, sebagaimana Timor Leste, me...",https://www.cnnindonesia.com/internasional/202...
1,Statistik Baggott Saat Ipswich Comeback Singki...,prabowo,CNN Indonesia,Sepakbola,"Rabu, 27 Sep 2023 08:14 WIB",Elkan Baggott kembali membantu Ipswich Town me...,https://www.cnnindonesia.com/olahraga/20230927...
2,Fakta-fakta Pembunuhan Imam Masykur Usai Rekon...,prabowo,CNN Indonesia,Hukum Kriminal,"Rabu, 27 Sep 2023 09:23 WIB",Sebanyak 23 adegan diperagakan di Pomdam Jaya ...,https://www.cnnindonesia.com/nasional/20230927...
3,Viral Polantas Bawa Motor Sambil Merokok terny...,prabowo,CNN Indonesia,Tren,"Rabu, 27 Sep 2023 07:09 WIB",Seorang polisi lalu lintas (polantas) yang vir...,https://www.cnnindonesia.com/otomotif/20230927...
4,"Deret Masalah di iPhone 15 Baru, Simak Daftarnya",prabowo,CNN Indonesia,e-Bisnis,"Rabu, 27 Sep 2023 09:02 WIB",iPhone 15 yang baru saja meluncur ternyata mem...,https://www.cnnindonesia.com/teknologi/2023092...
5,Title not found,prabowo,Author not found,Category not found,Date not found,Content not found,https://www.cnnindonesia.com/hiburan/202308311...
6,Title not found,prabowo,Author not found,Category not found,Date not found,Content not found,https://www.cnnindonesia.com/gaya-hidup/202309...
7,Pelatih Wolves Bingung Usai Kena Comeback Bagg...,prabowo,CNN Indonesia,Sepakbola,"Rabu, 27 Sep 2023 08:37 WIB",Pelatih Wolverhampton Wanderers Gary O'Neil di...,https://www.cnnindonesia.com/olahraga/20230927...
8,"Plaza Atrium Senen Dijual, Siapa Pemiliknya?",prabowo,CNN Indonesia,Bisnis,"Rabu, 27 Sep 2023 08:30 WIB",PT Cowell Development Tbk atau COWL resmi menj...,https://www.cnnindonesia.com/ekonomi/202309270...
9,Harga Minyak Naik Tipis Sentuh US$90 per Barel,prabowo,CNN Indonesia,Energi,"Rabu, 27 Sep 2023 08:59 WIB",Harga minyak naik tipis dalam perdagangan awal...,https://www.cnnindonesia.com/ekonomi/202309270...


In [15]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
filtered_df.head(10)

,title,keywords,author,category,date,content,link
15,Deretan Perwira Tinggi dan Menengah Dimutasi K...,prabowo,CNN Indonesia,Hukum Kriminal,"Rabu, 27 Sep 2023 08:48 WIB",Kapolri Jenderal Listyo Sigit Prabowo melakuka...,https://www.cnnindonesia.com/nasional/20230927...


Save Data

In [ ]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'./excel/cnn_{keywords}_{current_datetime}.xlsx'
# filtered_df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')